In [3]:
from Bio.PDB import *

import Bio.PDB as bpdb
from Bio.PDB.PDBIO import PDBIO
from Bio.PDB.DSSP import DSSP
from Bio.PDB.DSSP import dssp_dict_from_pdb_file
from Bio import SeqIO
from Bio import SearchIO
from Bio.SeqUtils import seq1
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

from Bio.PDB.Chain import Chain
from Bio.PDB.Atom import Atom
from Bio.PDB.Residue import Residue
from Bio.PDB.Model import Model

import pickle
import os
from os import path
from os.path import isfile, join, dirname, isdir, exists

import gudhi as gd
from scipy.optimize import minimize_scalar, least_squares

from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection
import numpy.linalg as linalg
from scipy import sparse
import scipy.signal
from scipy.optimize import minimize
from scipy.ndimage import gaussian_filter1d, gaussian_filter

import sys
import math

from scipy.optimize import curve_fit

import matplotlib.pyplot as plt

In [4]:
def get_files_with_ext(directory, ext):
    file_list = []
    for file in os.listdir(directory):
        if file.endswith(ext):
            file_list.append(file)
    return file_list

def get_geometries(directory):
    filenames = get_files_with_ext(directory, '.pdb')
    
    parser = PDBParser()
    geometries = {}
    for protein in filenames:
        name = os.path.splitext(protein)[0] # strip file extension
        chain = next(parser.get_structure('', os.path.join(directory, protein)).get_chains()) # 
        # chain = list(structure.get_chains())[0]
        geometries[name] = np.array([np.array(list(residue["CA"].get_vector())) for residue in chain.get_residues()])
        
    return geometries
        

In [5]:

A = get_geometries("nlrome_whole")

In [7]:
df = pd.read_csv('LRRPredictor/col-0-nlrome.txt', delimiter = '\t')

motifs_d = {}
for protid in A:
    motifs = []
    foundProtein = False
    for ii in range(len(df)):
        if '.'.join(df['annotation'][ii].split('_')[1:]) == protid:
            foundProtein = True
        elif foundProtein:            
            break
        if foundProtein:
            motifs.append(int(df['start'][ii]))
    motifs_d[protid] = motifs

# with open('pickles/motifs_d.pickle', 'wb') as handle:
#     pickle.dump(motifs_d, handle)        